In [4]:
import time

def plus1(u, v): #v will CHANGE to u+v; u won't change
    for k in u:
        if k in v:
            v[k] += u[k]
        else:
            v[k] = u[k]
            
def scal1(n, d): #d will CHANGE    
    for k in d:
        d[k] *= n

def concat(u, v): #u,v are dicts with xy-string/z-value-tuple keys; return uv
                    #watch out when u, v are empty dicts   
    r = {}
    for k in u:
        for l in v:           
            r[k + l] = u[k] * v[l] #k + l joins two strings or tuples
    return r

def l(u, v):
    r = concat(v,u)
    scal1(-1, r)
    plus1(concat(u,v), r)
    return r

class z:
    
    #PUT THE LAST PART INTO THE if m == ny PART
    #generate lyndon words of depth ny
    #the alphabet set = {1,2,3,...,nx=wt-dp}, order is 1 < 2 < 3 < ...
    def lyndon_words(nx, ny):

        #this part is added after the code check
        if nx == 0:
            if ny == 1:
                return [(1,)]
            else:
                return []
        if ny == 0:
            return []

        output = []
        w = [0]

        # Loop till w is empty 
        while w: 

            # Incrementing the last character 
            w[-1] += 1
            m = len(w) 
            if m == ny:
                #w is lyndon. Output it
                output.append(tuple(w)) #if w is not changed to tuple, will cause problems

            # Repeating w to get an ny-length string 
            while len(w) < ny: 
                w.append(w[-m]) 

            # Removing the last character 
            # as long it is equal to 
            # the largest letter in the alphabet set
            while w and w[-1] == nx: 
                w.pop()

        a = []
        for i in range(ny-1):
            a.append(1)
        a.append(nx+1)
        trimmed_output = [tuple(a)]

        #discard words not having weight wt0 = nx + ny
        wt0 = nx + ny    
        for i in range(len(output)):
            wt = 0
            for j in output[i]:
                wt += j

            if wt == wt0:
                trimmed_output.append(output[i])

        return trimmed_output #still follows the lexicographic order
### end of class z ###

def is_lyndon(t): #t: tuple; returns True if t is a letter
    for i in range(1, len(t)): #the last i is len(t)-1
        if t >= t[i:]: return False
    return True

def standard_fac(t): #t: lyndon tuple; returns nothing if t is a letter
    for i in range(1, len(t)):
        if is_lyndon(t[:i]) and is_lyndon(t[i:]):      
            return (t[:i], t[i:])
        
def lyn_lambda(t): #t: lyndon tuple
    if len(t) == 1:
        return t[0]
    
    t0, t1 = standard_fac(t)
    return [lyn_lambda(t0), lyn_lambda(t1)]

def lyn_lie_poly(t): #t: lyndon tuple
    if len(t) == 1:
        return {t: 1}
    
    t0, t1 = standard_fac(t)
    return l( lyn_lie_poly(t0) , lyn_lie_poly(t1) )

class xy:
    def lyndon_words(nx, ny):
        if nx == ny == 0: return []
        
        wt = nx + ny
        output = []
        w = [-1]

        # Loop till w is empty 
        while w: 

            # Incrementing the last character 
            w[-1] += 1
            m = len(w) 
            if m == wt:                
                #get the number of 1's in w
                ny_in_w = 0
                for i in w: ny_in_w += i
                
                if ny_in_w == ny:
                    output.append(tuple(w))

            # Repeating w to get a wt-length string 
            while len(w) < wt: 
                w.append(w[-m]) 

            # Removing the last character 
            # as long it is equal to 
            # the largest letter in the alphabet set
            while w and w[-1] == 1: 
                w.pop()
                
        return output
    
    def toz(d): #this is the projection map pi_y. 程式碼參考自mzv.py中的toz
        # e.g. k = (0,0,1,0,1,1,0,0,0,1). place of 1's: 2, 4, 5, 9. The result is (3,2,1,4) = (2+1,4-2,5-4,9-5). 

        r = {}
        for k in d:
            if k[-1] == 0: continue
            
            l = []
            #l records position of 1's
            for i in range(len(k)): #i=0 ~ len(k)-1
                if k[i] == 1:
                    l.append(i)

            ll = [ l[0] + 1 ]
            for i in range(0, len(l)-1): # 0 =< i =< (len(l)-1)-1 = len(l)-2
                ll.append(l[i+1] - l[i])

            t = tuple(ll)
            r[t] = d[k]
            
        return r

### end of class xy ###

def compare(nx, ny):

    lxy = xy.lyndon_words(nx, ny)
    lz = z.lyndon_words(nx, ny)
    llxy_before_toz = [ lyn_lie_poly( t ) for t in lxy ] #basis of the primitive elements of xy-shuffle (with nx x's and ny y's)
    llxy = [ xy.toz(lyn_lie_poly( t )) for t in lxy ]
    llz =  [ lyn_lie_poly( t ) for t in lz ] #basis of the primitive elements of z-value-shuffle (with nx x's and ny y's)
    
    print('primitive elements of xy-shuffle:')
    print(llxy_before_toz)
    print('pi_y( primitive elements of xy-shuffle ):')
    print(llxy)
    print('primitive elements of z-value-shuffle:')
    print(llz)
    
    ###################################
    
    terms = set()
    for d in llxy:
        for t in d:            
            terms.add(t)
    for d in llz:
        for t in d:
            terms.add(t)    
        
    terms_list = [ t for t in terms ]
    terms_list.sort()
    
    A = [] #matrix
    for d in llxy:
        v = [] #vector
        for i in range(0, len(terms_list)):
            if terms_list[i] in d:
                v.append( d[terms_list[i]] )
            else:
                v.append(0)
        A.append(v)
    print('Matrix of pi_y( primitive elements of xy-shuffle ):')
    print( Matrix(A) )
    
    B = []
    for d in llz:
        v = [] #vector
        for i in range(0, len(terms_list)):
            if terms_list[i] in d:
                v.append( d[terms_list[i]] )
            else:
                v.append(0)
        B.append(v)
    print('Matrix of primitive elements of z-value-shuffle:')
    print( Matrix(B) )
    
    for v in B: A.append(v)
    
    print('Kernel:')
    print( kernel(Matrix(A)) )
    
def find_nonzero_kernel(wt):
    t0 = time.time()
    
    for nx in range(1, wt-1): # 1 =< nx =< wt-2
        ny = wt - nx
        
        lxy = xy.lyndon_words(nx, ny)
        lz = z.lyndon_words(nx, ny)
        #llxy_before_toz = [ lyn_lie_poly( t ) for t in lxy ] #basis of the primitive elements of xy-shuffle (with nx x's and ny y's)
        llxy = [ xy.toz(lyn_lie_poly( t )) for t in lxy ]
        llz =  [ lyn_lie_poly( t ) for t in lz ] #basis of the primitive elements of z-value-shuffle (with nx x's and ny y's)

        ###################################

        terms = set()
        for d in llxy:
            for t in d:            
                terms.add(t)
        for d in llz:
            for t in d:
                terms.add(t)    

        terms_list = [ t for t in terms ]
        terms_list.sort()

        A = [] #matrix
        for d in llxy:
            v = [] #vector
            for i in range(0, len(terms_list)):
                if terms_list[i] in d:
                    v.append( d[terms_list[i]] )
                else:
                    v.append(0)
            A.append(v)
        #print('Matrix of pi_y( primitive elements of xy-shuffle ):')
        #print( Matrix(A) )

        B = []
        for d in llz:
            v = [] #vector
            for i in range(0, len(terms_list)):
                if terms_list[i] in d:
                    v.append( d[terms_list[i]] )
                else:
                    v.append(0)
            B.append(v)
        #print('Matrix of primitive elements of z-value-shuffle:')
        #print( Matrix(B) )

        for v in B: A.append(v)

        print('(nx, ny) = {}:'.format((nx, ny)))
        print( kernel(Matrix(A)) )
    
    t1 = time.time()
    print('Computation time: {}'.format(t1 - t0))
    
class ls:
    def __init__(self, nx, ny):
    # .nx .ny .llxy .llz
        self.nx = nx
        self.ny = ny        
        
        lxy = xy.lyndon_words(nx, ny)
        lz = z.lyndon_words(nx, ny)
        self.llxy = [ xy.toz(lyn_lie_poly( t )) for t in lxy ] #pi_y( basis of the primitive elements of xy-value-shuffle )
        self.llz =  [ lyn_lie_poly( t ) for t in lz ] #basis of primitive elements of z-value-shuffle
        
    # .terms
        s = set()
        for d in self.llxy:
            for t in d:            
                s.add(t)
        for d in self.llz:
            for t in d:
                s.add(t)    

        s_list = [ t for t in s ]
        s_list.sort()        
        self.terms = s_list
    
    # .xy_matrix
    # Matrix of pi_y( primitive elements of xy-shuffle )
        A = []
        terms_list = self.terms
        
        for d in self.llxy:
            v = [] #row vector
            for i in range(0, len(terms_list)):
                if terms_list[i] in d:
                    v.append( d[terms_list[i]] )
                else:
                    v.append(0)
            A.append(v)

        self.xy_matrix = Matrix(A)
        
    # .z_matrix
    # Matrix of primitive elements of z-value-shuffle
        B = []        
        
        for d in self.llz:
            v = []
            for i in range(0, len(terms_list)):
                if terms_list[i] in d:
                    v.append( d[terms_list[i]] )
                else:
                    v.append(0)
            B.append(v)
        
        self.z_matrix = Matrix(B)
        
    # .kernel
        for v in B: A.append(v)
        self.kernel = kernel(Matrix(A))

### end of class ls ###
        
        

In [2]:
compare(3,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0): 1, (1, 0, 1, 0, 0): -2, (0, 1, 1, 0, 0): -1, (0, 1, 0, 1, 0): 4, (0, 0, 1, 1, 0): -1, (0, 0, 1, 0, 1): -2, (0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0): 1, (1, 0, 0, 1, 0): -3, (1, 0, 0, 0, 1): 2, (0, 1, 1, 0, 0): -1, (0, 1, 0, 1, 0): 4, (0, 1, 0, 0, 1): -3, (0, 0, 1, 1, 0): -1, (0, 0, 1, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(3, 2): -2, (4, 1): 1}, {(1, 4): 2, (2, 3): -3, (3, 2): 1}]
primitive elements of z-value-shuffle:
[{(4, 1): -1, (1, 4): 1}, {(3, 2): -1, (2, 3): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[ 0  0 -2  1]
[ 2 -3  1  0]
Matrix of primitive elements of z-value-shuffle:
[ 1  0  0 -1]
[ 0  1 -1  0]
Kernel:
Free module of degree 4 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [9]:
A = [[0,-2,0,1],
     [-3,1,2,0],
     [0,0,1,-1],
     [1,-1,0,0]]
A = Matrix(A)
zero_vec = vector([0,0,0,0])
print(A.solve_left(zero_vec))
print(kernel(A))

(0, 0, 0, 0)
Free module of degree 4 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [57]:
compare(2,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0): 1, (1, 1, 0, 1, 0): -3, (1, 0, 1, 1, 0): 3, (0, 1, 1, 1, 0): -2, (0, 1, 1, 0, 1): 3, (0, 1, 0, 1, 1): -3, (0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0): 1, (1, 1, 0, 0, 1): -1, (1, 0, 1, 1, 0): -3, (1, 0, 1, 0, 1): 4, (0, 1, 1, 1, 0): 2, (0, 1, 1, 0, 1): -3, (1, 0, 0, 1, 1): -1, (0, 1, 0, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(2, 1, 2): 3, (2, 2, 1): -3, (3, 1, 1): 1}, {(1, 1, 3): -1, (1, 2, 2): 4, (2, 1, 2): -3, (1, 3, 1): -1, (2, 2, 1): 1}]
primitive elements of z-value-shuffle:
[{(3, 1, 1): 1, (1, 3, 1): -2, (1, 1, 3): 1}, {(2, 2, 1): 1, (2, 1, 2): -2, (1, 2, 2): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[ 0  0  0  3 -3  1]
[-1  4 -1 -3  1  0]
Matrix of primitive elements of z-value-shuffle:
[ 1  0 -2  0  0  1]
[ 0  1  0 -2  1  0]
Kernel:
Free module of degree 4 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [14]:
A = [[0,0,1,0,3,-3],
     [-1,-1,0,4,-3,1],
     [1,-2,1,0,0,0],
     [0,0,0,1,-2,1]]
A = Matrix(A)
print(A)
kernel(A)

[ 0  0  1  0  3 -3]
[-1 -1  0  4 -3  1]
[ 1 -2  1  0  0  0]
[ 0  0  0  1 -2  1]


Free module of degree 4 and rank 0 over Integer Ring
Echelon basis matrix:
[]

In [58]:
compare(4,2) #(3,3)這一項，primitive elements of z-value shuffle 裡沒有
#又無解

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0): 2, (0, 1, 0, 1, 0, 0): -6, (0, 0, 1, 1, 0, 0): 0, (0, 0, 1, 0, 1, 0): 6, (0, 0, 0, 1, 1, 0): -2, (0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0): -1, (1, 0, 0, 1, 0, 0): 3, (1, 0, 0, 0, 1, 0): -2, (0, 1, 1, 0, 0, 0): 1, (0, 1, 0, 1, 0, 0): -3, (0, 1, 0, 0, 1, 0): 0, (0, 0, 1, 1, 0, 0): 0, (0, 0, 1, 0, 1, 0): 3, (0, 1, 0, 0, 0, 1): 2, (0, 0, 1, 0, 0, 1): -3, (0, 0, 0, 1, 1, 0): -1, (0, 0, 0, 1, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(4, 2): -2, (5, 1): 1}, {(2, 4): 2, (3, 3): -3, (4, 2): 1}]
primitive elements of z-value-shuffle:
[{(5, 1): -1, (1, 5): 1}, {(4, 2): -1, (2, 4): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[ 0  0  0 -2  1]
[ 0  2 -3  1  0]
Matrix of primitive elements of z-value-shuffle:
[ 1  0  0  0 -1]
[ 0  1  0 -1  0]
Kernel:
Free module of degree 4 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [3]:
compare(3,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0, 0): -1, (1, 1, 0, 1, 0, 0): 3, (1, 0, 1, 1, 0, 0): -3, (0, 1, 1, 1, 0, 0): 3, (0, 1, 1, 0, 1, 0): -6, (0, 1, 0, 1, 1, 0): 6, (0, 0, 1, 1, 1, 0): -3, (0, 0, 1, 1, 0, 1): 3, (0, 0, 1, 0, 1, 1): -3, (0, 0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0, 0): -1, (1, 1, 0, 0, 1, 0): 1, (1, 0, 1, 1, 0, 0): 3, (1, 0, 1, 0, 1, 0): -4, (0, 1, 1, 1, 0, 0): -2, (0, 1, 1, 0, 1, 0): 4, (1, 0, 0, 1, 1, 0): 1, (0, 1, 0, 1, 1, 0): -4, (0, 1, 1, 0, 0, 1): -1, (0, 1, 0, 1, 0, 1): 4, (0, 0, 1, 1, 1, 0): 2, (0, 0, 1, 1, 0, 1): -3, (0, 1, 0, 0, 1, 1): -1, (0, 0, 1, 0, 1, 1): 1}, {(1, 0, 1, 1, 0, 0): -1, (1, 0, 1, 0, 1, 0): 0, (1, 0, 0, 1, 1, 0): 0, (1, 0, 0, 1, 0, 1): -2, (1, 0, 0, 0, 1, 1): 1, (0, 1, 1, 1, 0, 0): 1, (0, 1, 1, 0, 1, 0): -2, (0, 1, 0, 1, 1, 0): 2, (0, 1, 0, 1, 0, 1): 0, (0, 1, 0, 0, 1, 1): -1, (1, 1, 0, 0, 1, 0): 1, (1, 1, 0, 0, 0, 1): -1, (1, 0, 1, 0, 0, 1): 2, (0, 1, 1, 0, 0, 1): 0, (0, 0, 1, 1, 1, 0): -1, (0, 0, 1, 1, 0, 1): 1}]
pi_y( primitive eleme

In [60]:
compare(2,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0, 0): -1, (1, 1, 1, 0, 1, 0): 4, (1, 1, 0, 1, 1, 0): -6, (1, 0, 1, 1, 1, 0): 4, (0, 1, 1, 1, 1, 0): 0, (0, 1, 1, 1, 0, 1): -4, (0, 1, 1, 0, 1, 1): 6, (0, 1, 0, 1, 1, 1): -4, (0, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 0): -1, (1, 1, 1, 0, 0, 1): 1, (1, 1, 0, 1, 1, 0): 3, (1, 1, 0, 1, 0, 1): -3, (1, 0, 1, 1, 1, 0): -2, (1, 0, 1, 1, 0, 1): 0, (0, 1, 1, 1, 1, 0): 0, (0, 1, 1, 1, 0, 1): 2, (1, 0, 1, 0, 1, 1): 3, (1, 0, 0, 1, 1, 1): -1, (0, 1, 1, 0, 1, 1): -3, (0, 1, 0, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(2, 1, 1, 2): -4, (2, 1, 2, 1): 6, (2, 2, 1, 1): -4, (3, 1, 1, 1): 1}, {(1, 1, 1, 3): 1, (1, 1, 2, 2): -3, (1, 2, 1, 2): 0, (2, 1, 1, 2): 2, (1, 2, 2, 1): 3, (1, 3, 1, 1): -1, (2, 1, 2, 1): -3, (2, 2, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(3, 1, 1, 1): -1, (1, 3, 1, 1): 3, (1, 1, 3, 1): -3, (1, 1, 1, 3): 1}, {(2, 2, 1, 1): -1, (2, 1, 2, 1): 2, (1, 2, 2, 1): 0, (1, 2, 1, 2): -2, (1, 1, 2, 2): 1}]
Matrix of pi

In [61]:
compare(1,5)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 0): -1, (1, 1, 1, 1, 0, 1): 5, (1, 1, 1, 0, 1, 1): -10, (1, 1, 0, 1, 1, 1): 10, (1, 0, 1, 1, 1, 1): -5, (0, 1, 1, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 1, 1, 2): 5, (1, 1, 1, 2, 1): -10, (1, 1, 2, 1, 1): 10, (1, 2, 1, 1, 1): -5, (2, 1, 1, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1, 1, 1): 1, (1, 2, 1, 1, 1): -4, (1, 1, 2, 1, 1): 6, (1, 1, 1, 2, 1): -4, (1, 1, 1, 1, 2): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[  5 -10  10  -5   1]
Matrix of primitive elements of z-value-shuffle:
[ 1 -4  6 -4  1]
Kernel:
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [70]:
compare(5,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0): 1, (1, 0, 1, 0, 0, 0, 0): -2, (0, 1, 1, 0, 0, 0, 0): -3, (0, 1, 0, 1, 0, 0, 0): 8, (0, 0, 1, 1, 0, 0, 0): 2, (0, 0, 1, 0, 1, 0, 0): -12, (0, 0, 0, 1, 1, 0, 0): 2, (0, 0, 0, 1, 0, 1, 0): 8, (0, 0, 0, 0, 1, 1, 0): -3, (0, 0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0, 0): 1, (1, 0, 0, 1, 0, 0, 0): -3, (1, 0, 0, 0, 1, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0): -1, (0, 1, 0, 1, 0, 0, 0): 2, (0, 1, 0, 0, 1, 0, 0): 3, (0, 0, 1, 1, 0, 0, 0): 1, (0, 0, 1, 0, 1, 0, 0): -6, (0, 1, 0, 0, 0, 1, 0): -4, (0, 0, 1, 0, 0, 1, 0): 3, (0, 0, 0, 1, 1, 0, 0): 1, (0, 0, 0, 1, 0, 1, 0): 2, (0, 0, 1, 0, 0, 0, 1): 2, (0, 0, 0, 1, 0, 0, 1): -3, (0, 0, 0, 0, 1, 1, 0): -1, (0, 0, 0, 0, 1, 0, 1): 1}, {(1, 0, 0, 1, 0, 0, 0): 1, (1, 0, 0, 0, 1, 0, 0): -4, (1, 0, 0, 0, 0, 1, 0): 5, (1, 0, 0, 0, 0, 0, 1): -2, (0, 1, 0, 1, 0, 0, 0): -2, (0, 1, 0, 0, 1, 0, 0): 9, (0, 1, 0, 0, 0, 1, 0): -12, (0, 1, 0, 0, 0, 0, 1): 5, (0, 0, 1, 1, 0, 0, 0): 1, (0, 0, 

In [65]:
compare(4,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0, 0, 0): 1, (1, 1, 0, 1, 0, 0, 0): -3, (1, 0, 1, 1, 0, 0, 0): 3, (0, 1, 1, 1, 0, 0, 0): -4, (0, 1, 1, 0, 1, 0, 0): 9, (0, 1, 0, 1, 1, 0, 0): -9, (0, 0, 1, 1, 1, 0, 0): 6, (0, 0, 1, 1, 0, 1, 0): -9, (0, 0, 1, 0, 1, 1, 0): 9, (0, 0, 0, 1, 1, 1, 0): -4, (0, 0, 0, 1, 1, 0, 1): 3, (0, 0, 0, 1, 0, 1, 1): -3, (0, 0, 0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0, 0, 0): 1, (1, 1, 0, 0, 1, 0, 0): -1, (1, 0, 1, 1, 0, 0, 0): -3, (1, 0, 1, 0, 1, 0, 0): 4, (0, 1, 1, 1, 0, 0, 0): 2, (0, 1, 1, 0, 1, 0, 0): -5, (1, 0, 0, 1, 1, 0, 0): -1, (0, 1, 0, 1, 1, 0, 0): 7, (0, 1, 1, 0, 0, 1, 0): 2, (0, 1, 0, 1, 0, 1, 0): -8, (0, 0, 1, 1, 1, 0, 0): -4, (0, 0, 1, 1, 0, 1, 0): 7, (0, 1, 0, 0, 1, 1, 0): 2, (0, 0, 1, 0, 1, 1, 0): -5, (0, 0, 1, 1, 0, 0, 1): -1, (0, 0, 1, 0, 1, 0, 1): 4, (0, 0, 0, 1, 1, 1, 0): 2, (0, 0, 0, 1, 1, 0, 1): -3, (0, 0, 1, 0, 0, 1, 1): -1, (0, 0, 0, 1, 0, 1, 1): 1}, {(1, 0, 1, 1, 0, 0, 0): 1, (1, 0, 1, 0, 1, 0, 0): 0, (1, 0, 0, 1, 1, 0, 0): 0, (1, 0, 0

In [66]:
compare(3,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0, 0, 0): 1, (1, 1, 1, 0, 1, 0, 0): -4, (1, 1, 0, 1, 1, 0, 0): 6, (1, 0, 1, 1, 1, 0, 0): -4, (0, 1, 1, 1, 1, 0, 0): -1, (0, 1, 1, 1, 0, 1, 0): 8, (0, 1, 1, 0, 1, 1, 0): -12, (0, 1, 0, 1, 1, 1, 0): 8, (0, 0, 1, 1, 1, 1, 0): -1, (0, 0, 1, 1, 1, 0, 1): -4, (0, 0, 1, 1, 0, 1, 1): 6, (0, 0, 1, 0, 1, 1, 1): -4, (0, 0, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 0, 0): 1, (1, 1, 1, 0, 0, 1, 0): -1, (1, 1, 0, 1, 1, 0, 0): -3, (1, 1, 0, 1, 0, 1, 0): 3, (1, 0, 1, 1, 1, 0, 0): 2, (1, 0, 1, 1, 0, 1, 0): 0, (0, 1, 1, 1, 1, 0, 0): 0, (0, 1, 1, 1, 0, 1, 0): -3, (1, 0, 1, 0, 1, 1, 0): -3, (1, 0, 0, 1, 1, 1, 0): 1, (0, 1, 1, 0, 1, 1, 0): 6, (0, 1, 0, 1, 1, 1, 0): -3, (0, 1, 1, 1, 0, 0, 1): 1, (0, 1, 1, 0, 1, 0, 1): -3, (0, 1, 0, 1, 1, 0, 1): 0, (0, 0, 1, 1, 1, 1, 0): 0, (0, 0, 1, 1, 1, 0, 1): 2, (0, 1, 0, 1, 0, 1, 1): 3, (0, 1, 0, 0, 1, 1, 1): -1, (0, 0, 1, 1, 0, 1, 1): -3, (0, 0, 1, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 1, 0, 0): 1, (1, 1, 0, 1, 0, 1, 0): -2, (1, 1, 

In [67]:
compare(2,5)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 0, 0): 1, (1, 1, 1, 1, 0, 1, 0): -5, (1, 1, 1, 0, 1, 1, 0): 10, (1, 1, 0, 1, 1, 1, 0): -10, (1, 0, 1, 1, 1, 1, 0): 5, (0, 1, 1, 1, 1, 1, 0): -2, (0, 1, 1, 1, 1, 0, 1): 5, (0, 1, 1, 1, 0, 1, 1): -10, (0, 1, 1, 0, 1, 1, 1): 10, (0, 1, 0, 1, 1, 1, 1): -5, (0, 0, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 0, 1, 0): 1, (1, 1, 1, 1, 0, 0, 1): -1, (1, 1, 1, 0, 1, 1, 0): -4, (1, 1, 1, 0, 1, 0, 1): 4, (1, 1, 0, 1, 1, 1, 0): 6, (1, 1, 0, 1, 1, 0, 1): -6, (1, 0, 1, 1, 1, 1, 0): -5, (1, 0, 1, 1, 1, 0, 1): 8, (0, 1, 1, 1, 1, 1, 0): 2, (0, 1, 1, 1, 1, 0, 1): -5, (1, 0, 1, 1, 0, 1, 1): -6, (1, 0, 1, 0, 1, 1, 1): 4, (1, 0, 0, 1, 1, 1, 1): -1, (0, 1, 1, 1, 0, 1, 1): 6, (0, 1, 1, 0, 1, 1, 1): -4, (0, 1, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 1, 0): 1, (1, 1, 1, 0, 1, 0, 1): -2, (1, 1, 1, 0, 0, 1, 1): 1, (1, 1, 0, 1, 1, 1, 0): -4, (1, 1, 0, 1, 1, 0, 1): 9, (1, 1, 0, 1, 0, 1, 1): -6, (1, 0, 1, 1, 1, 1, 0): 5, (1, 0, 1, 1, 1, 0, 1): -12, (1, 0, 1, 1, 0, 1, 1): 9, (0

In [68]:
compare(1,6)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 0): 1, (1, 1, 1, 1, 1, 0, 1): -6, (1, 1, 1, 1, 0, 1, 1): 15, (1, 1, 1, 0, 1, 1, 1): -20, (1, 1, 0, 1, 1, 1, 1): 15, (1, 0, 1, 1, 1, 1, 1): -6, (0, 1, 1, 1, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 1, 1, 1, 2): -6, (1, 1, 1, 1, 2, 1): 15, (1, 1, 1, 2, 1, 1): -20, (1, 1, 2, 1, 1, 1): 15, (1, 2, 1, 1, 1, 1): -6, (2, 1, 1, 1, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1, 1, 1, 1): -1, (1, 2, 1, 1, 1, 1): 5, (1, 1, 2, 1, 1, 1): -10, (1, 1, 1, 2, 1, 1): 10, (1, 1, 1, 1, 2, 1): -5, (1, 1, 1, 1, 1, 2): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[ -6  15 -20  15  -6   1]
Matrix of primitive elements of z-value-shuffle:
[  1  -5  10 -10   5  -1]
Kernel:
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [4]:
compare(6,2)
#test push-invariance

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0): 4, (0, 1, 0, 1, 0, 0, 0, 0): -10, (0, 0, 1, 1, 0, 0, 0, 0): -5, (0, 0, 1, 0, 1, 0, 0, 0): 20, (0, 0, 0, 1, 1, 0, 0, 0): 0, (0, 0, 0, 1, 0, 1, 0, 0): -20, (0, 0, 0, 0, 1, 1, 0, 0): 5, (0, 0, 0, 0, 1, 0, 1, 0): 10, (0, 0, 0, 0, 0, 1, 1, 0): -4, (0, 0, 0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0, 0, 0): -1, (1, 0, 0, 1, 0, 0, 0, 0): 3, (1, 0, 0, 0, 1, 0, 0, 0): -2, (0, 1, 1, 0, 0, 0, 0, 0): 1, (0, 1, 0, 1, 0, 0, 0, 0): -1, (0, 1, 0, 0, 1, 0, 0, 0): -6, (0, 0, 1, 1, 0, 0, 0, 0): -2, (0, 0, 1, 0, 1, 0, 0, 0): 8, (0, 1, 0, 0, 0, 1, 0, 0): 6, (0, 0, 1, 0, 0, 1, 0, 0): 0, (0, 0, 0, 1, 1, 0, 0, 0): 0, (0, 0, 0, 1, 0, 1, 0, 0): -8, (0, 0, 1, 0, 0, 0, 1, 0): -6, (0, 0, 0, 1, 0, 0, 1, 0): 6, (0, 0, 0, 0, 1, 1, 0, 0): 2, (0, 0, 0, 0, 1, 0, 1, 0): 1, (0, 0, 0, 1, 0, 0, 0, 1): 2, (0, 0, 0, 0, 1, 0, 0, 1): -3, (0, 0, 0, 0, 0, 1, 1, 0): -1, (0, 0, 0, 0, 0,

In [72]:
compare(5,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0, 0, 0, 0): -1, (1, 1, 0, 1, 0, 0, 0, 0): 3, (1, 0, 1, 1, 0, 0, 0, 0): -3, (0, 1, 1, 1, 0, 0, 0, 0): 5, (0, 1, 1, 0, 1, 0, 0, 0): -12, (0, 1, 0, 1, 1, 0, 0, 0): 12, (0, 0, 1, 1, 1, 0, 0, 0): -10, (0, 0, 1, 1, 0, 1, 0, 0): 18, (0, 0, 1, 0, 1, 1, 0, 0): -18, (0, 0, 0, 1, 1, 1, 0, 0): 10, (0, 0, 0, 1, 1, 0, 1, 0): -12, (0, 0, 0, 1, 0, 1, 1, 0): 12, (0, 0, 0, 0, 1, 1, 1, 0): -5, (0, 0, 0, 0, 1, 1, 0, 1): 3, (0, 0, 0, 0, 1, 0, 1, 1): -3, (0, 0, 0, 0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0, 0, 0, 0): -1, (1, 1, 0, 0, 1, 0, 0, 0): 1, (1, 0, 1, 1, 0, 0, 0, 0): 3, (1, 0, 1, 0, 1, 0, 0, 0): -4, (0, 1, 1, 1, 0, 0, 0, 0): -2, (0, 1, 1, 0, 1, 0, 0, 0): 6, (1, 0, 0, 1, 1, 0, 0, 0): 1, (0, 1, 0, 1, 1, 0, 0, 0): -10, (0, 1, 1, 0, 0, 1, 0, 0): -3, (0, 1, 0, 1, 0, 1, 0, 0): 12, (0, 0, 1, 1, 1, 0, 0, 0): 6, (0, 0, 1, 1, 0, 1, 0, 0): -12, (0, 1, 0, 0, 1, 1, 0, 0): -3, (0, 0, 1, 0, 1, 1, 0, 0): 12, (0, 0, 1, 1, 0, 0, 1, 0): 3, (0, 0, 1, 0, 1, 0, 1, 0): -12, (0, 0

In [73]:
compare(4,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0, 0, 0, 0): -1, (1, 1, 1, 0, 1, 0, 0, 0): 4, (1, 1, 0, 1, 1, 0, 0, 0): -6, (1, 0, 1, 1, 1, 0, 0, 0): 4, (0, 1, 1, 1, 1, 0, 0, 0): 2, (0, 1, 1, 1, 0, 1, 0, 0): -12, (0, 1, 1, 0, 1, 1, 0, 0): 18, (0, 1, 0, 1, 1, 1, 0, 0): -12, (0, 0, 1, 1, 1, 1, 0, 0): 0, (0, 0, 1, 1, 1, 0, 1, 0): 12, (0, 0, 1, 1, 0, 1, 1, 0): -18, (0, 0, 1, 0, 1, 1, 1, 0): 12, (0, 0, 0, 1, 1, 1, 1, 0): -2, (0, 0, 0, 1, 1, 1, 0, 1): -4, (0, 0, 0, 1, 1, 0, 1, 1): 6, (0, 0, 0, 1, 0, 1, 1, 1): -4, (0, 0, 0, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 0, 0, 0): -1, (1, 1, 1, 0, 0, 1, 0, 0): 1, (1, 1, 0, 1, 1, 0, 0, 0): 3, (1, 1, 0, 1, 0, 1, 0, 0): -3, (1, 0, 1, 1, 1, 0, 0, 0): -2, (1, 0, 1, 1, 0, 1, 0, 0): 0, (0, 1, 1, 1, 1, 0, 0, 0): 0, (0, 1, 1, 1, 0, 1, 0, 0): 4, (1, 0, 1, 0, 1, 1, 0, 0): 3, (1, 0, 0, 1, 1, 1, 0, 0): -1, (0, 1, 1, 0, 1, 1, 0, 0): -9, (0, 1, 0, 1, 1, 1, 0, 0): 5, (0, 1, 1, 1, 0, 0, 1, 0): -2, (0, 1, 1, 0, 1, 0, 1, 0): 6, (0, 1, 0, 1, 1, 0, 1, 0): 0, (0, 0, 1, 1, 1

In [74]:
compare(3,5)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 0, 0, 0): -1, (1, 1, 1, 1, 0, 1, 0, 0): 5, (1, 1, 1, 0, 1, 1, 0, 0): -10, (1, 1, 0, 1, 1, 1, 0, 0): 10, (1, 0, 1, 1, 1, 1, 0, 0): -5, (0, 1, 1, 1, 1, 1, 0, 0): 3, (0, 1, 1, 1, 1, 0, 1, 0): -10, (0, 1, 1, 1, 0, 1, 1, 0): 20, (0, 1, 1, 0, 1, 1, 1, 0): -20, (0, 1, 0, 1, 1, 1, 1, 0): 10, (0, 0, 1, 1, 1, 1, 1, 0): -3, (0, 0, 1, 1, 1, 1, 0, 1): 5, (0, 0, 1, 1, 1, 0, 1, 1): -10, (0, 0, 1, 1, 0, 1, 1, 1): 10, (0, 0, 1, 0, 1, 1, 1, 1): -5, (0, 0, 0, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 0, 1, 0, 0): -1, (1, 1, 1, 1, 0, 0, 1, 0): 1, (1, 1, 1, 0, 1, 1, 0, 0): 4, (1, 1, 1, 0, 1, 0, 1, 0): -4, (1, 1, 0, 1, 1, 1, 0, 0): -6, (1, 1, 0, 1, 1, 0, 1, 0): 6, (1, 0, 1, 1, 1, 1, 0, 0): 5, (1, 0, 1, 1, 1, 0, 1, 0): -8, (0, 1, 1, 1, 1, 1, 0, 0): -2, (0, 1, 1, 1, 1, 0, 1, 0): 6, (1, 0, 1, 1, 0, 1, 1, 0): 6, (1, 0, 1, 0, 1, 1, 1, 0): -4, (1, 0, 0, 1, 1, 1, 1, 0): 1, (0, 1, 1, 1, 0, 1, 1, 0): -10, (0, 1, 1, 0, 1, 1, 1, 0): 10, (0, 1, 0, 1, 1, 1, 1, 0): -6, (0, 1, 1

In [75]:
compare(2,6)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 0, 0): -1, (1, 1, 1, 1, 1, 0, 1, 0): 6, (1, 1, 1, 1, 0, 1, 1, 0): -15, (1, 1, 1, 0, 1, 1, 1, 0): 20, (1, 1, 0, 1, 1, 1, 1, 0): -15, (1, 0, 1, 1, 1, 1, 1, 0): 6, (0, 1, 1, 1, 1, 1, 1, 0): 0, (0, 1, 1, 1, 1, 1, 0, 1): -6, (0, 1, 1, 1, 1, 0, 1, 1): 15, (0, 1, 1, 1, 0, 1, 1, 1): -20, (0, 1, 1, 0, 1, 1, 1, 1): 15, (0, 1, 0, 1, 1, 1, 1, 1): -6, (0, 0, 1, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 1, 0, 1, 0): -1, (1, 1, 1, 1, 1, 0, 0, 1): 1, (1, 1, 1, 1, 0, 1, 1, 0): 5, (1, 1, 1, 1, 0, 1, 0, 1): -5, (1, 1, 1, 0, 1, 1, 1, 0): -10, (1, 1, 1, 0, 1, 1, 0, 1): 10, (1, 1, 0, 1, 1, 1, 1, 0): 10, (1, 1, 0, 1, 1, 1, 0, 1): -10, (1, 0, 1, 1, 1, 1, 1, 0): -4, (1, 0, 1, 1, 1, 1, 0, 1): 0, (0, 1, 1, 1, 1, 1, 1, 0): 0, (0, 1, 1, 1, 1, 1, 0, 1): 4, (1, 0, 1, 1, 1, 0, 1, 1): 10, (1, 0, 1, 1, 0, 1, 1, 1): -10, (1, 0, 1, 0, 1, 1, 1, 1): 5, (1, 0, 0, 1, 1, 1, 1, 1): -1, (0, 1, 1, 1, 1, 0, 1, 1): -10, (0, 1, 1, 1, 0, 1, 1, 1): 10, (0, 1, 1, 0, 1, 1, 1, 1): -5, (0, 1

In [76]:
compare(1,7)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 1, 0): -1, (1, 1, 1, 1, 1, 1, 0, 1): 7, (1, 1, 1, 1, 1, 0, 1, 1): -21, (1, 1, 1, 1, 0, 1, 1, 1): 35, (1, 1, 1, 0, 1, 1, 1, 1): -35, (1, 1, 0, 1, 1, 1, 1, 1): 21, (1, 0, 1, 1, 1, 1, 1, 1): -7, (0, 1, 1, 1, 1, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 1, 1, 1, 1, 2): 7, (1, 1, 1, 1, 1, 2, 1): -21, (1, 1, 1, 1, 2, 1, 1): 35, (1, 1, 1, 2, 1, 1, 1): -35, (1, 1, 2, 1, 1, 1, 1): 21, (1, 2, 1, 1, 1, 1, 1): -7, (2, 1, 1, 1, 1, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1, 1, 1, 1, 1): 1, (1, 2, 1, 1, 1, 1, 1): -6, (1, 1, 2, 1, 1, 1, 1): 15, (1, 1, 1, 2, 1, 1, 1): -20, (1, 1, 1, 1, 2, 1, 1): 15, (1, 1, 1, 1, 1, 2, 1): -6, (1, 1, 1, 1, 1, 1, 2): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[  7 -21  35 -35  21  -7   1]
Matrix of primitive elements of z-value-shuffle:
[  1  -6  15 -20  15  -6   1]
Kernel:
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]


In [78]:
compare(7,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0): 1, (1, 0, 1, 0, 0, 0, 0, 0, 0): -2, (0, 1, 1, 0, 0, 0, 0, 0, 0): -5, (0, 1, 0, 1, 0, 0, 0, 0, 0): 12, (0, 0, 1, 1, 0, 0, 0, 0, 0): 9, (0, 0, 1, 0, 1, 0, 0, 0, 0): -30, (0, 0, 0, 1, 1, 0, 0, 0, 0): -5, (0, 0, 0, 1, 0, 1, 0, 0, 0): 40, (0, 0, 0, 0, 1, 1, 0, 0, 0): -5, (0, 0, 0, 0, 1, 0, 1, 0, 0): -30, (0, 0, 0, 0, 0, 1, 1, 0, 0): 9, (0, 0, 0, 0, 0, 1, 0, 1, 0): 12, (0, 0, 0, 0, 0, 0, 1, 1, 0): -5, (0, 0, 0, 0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0, 0, 0, 0): 1, (1, 0, 0, 1, 0, 0, 0, 0, 0): -3, (1, 0, 0, 0, 1, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0): -1, (0, 1, 0, 1, 0, 0, 0, 0, 0): 0, (0, 1, 0, 0, 1, 0, 0, 0, 0): 9, (0, 0, 1, 1, 0, 0, 0, 0, 0): 3, (0, 0, 1, 0, 1, 0, 0, 0, 0): -9, (0, 1, 0, 0, 0, 1, 0, 0, 0): -8, (0, 0, 1, 0, 0, 1, 0, 0, 0): -6, (0, 0, 0, 1, 1, 0, 0, 0, 0): -2, (0, 0, 0, 1, 0, 1, 0, 0, 0): 16, (0, 0, 1, 0, 0, 0, 1, 0, 0): 12, (0, 0, 0, 1, 0, 0, 1, 0, 0): -6, (0, 0, 0, 0,

In [79]:
compare(6,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0, 0, 0, 0, 0): 1, (1, 1, 0, 1, 0, 0, 0, 0, 0): -3, (1, 0, 1, 1, 0, 0, 0, 0, 0): 3, (0, 1, 1, 1, 0, 0, 0, 0, 0): -6, (0, 1, 1, 0, 1, 0, 0, 0, 0): 15, (0, 1, 0, 1, 1, 0, 0, 0, 0): -15, (0, 0, 1, 1, 1, 0, 0, 0, 0): 15, (0, 0, 1, 1, 0, 1, 0, 0, 0): -30, (0, 0, 1, 0, 1, 1, 0, 0, 0): 30, (0, 0, 0, 1, 1, 1, 0, 0, 0): -20, (0, 0, 0, 1, 1, 0, 1, 0, 0): 30, (0, 0, 0, 1, 0, 1, 1, 0, 0): -30, (0, 0, 0, 0, 1, 1, 1, 0, 0): 15, (0, 0, 0, 0, 1, 1, 0, 1, 0): -15, (0, 0, 0, 0, 1, 0, 1, 1, 0): 15, (0, 0, 0, 0, 0, 1, 1, 1, 0): -6, (0, 0, 0, 0, 0, 1, 1, 0, 1): 3, (0, 0, 0, 0, 0, 1, 0, 1, 1): -3, (0, 0, 0, 0, 0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0, 0, 0, 0, 0): 1, (1, 1, 0, 0, 1, 0, 0, 0, 0): -1, (1, 0, 1, 1, 0, 0, 0, 0, 0): -3, (1, 0, 1, 0, 1, 0, 0, 0, 0): 4, (0, 1, 1, 1, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 1, 0, 0, 0, 0): -7, (1, 0, 0, 1, 1, 0, 0, 0, 0): -1, (0, 1, 0, 1, 1, 0, 0, 0, 0): 13, (0, 1, 1, 0, 0, 1, 0, 0, 0): 4, (0, 1, 0, 1, 0, 1, 0, 0, 0): -16, (0, 0, 

In [80]:
compare(5,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0, 0, 0, 0, 0): 1, (1, 1, 1, 0, 1, 0, 0, 0, 0): -4, (1, 1, 0, 1, 1, 0, 0, 0, 0): 6, (1, 0, 1, 1, 1, 0, 0, 0, 0): -4, (0, 1, 1, 1, 1, 0, 0, 0, 0): -3, (0, 1, 1, 1, 0, 1, 0, 0, 0): 16, (0, 1, 1, 0, 1, 1, 0, 0, 0): -24, (0, 1, 0, 1, 1, 1, 0, 0, 0): 16, (0, 0, 1, 1, 1, 1, 0, 0, 0): 2, (0, 0, 1, 1, 1, 0, 1, 0, 0): -24, (0, 0, 1, 1, 0, 1, 1, 0, 0): 36, (0, 0, 1, 0, 1, 1, 1, 0, 0): -24, (0, 0, 0, 1, 1, 1, 1, 0, 0): 2, (0, 0, 0, 1, 1, 1, 0, 1, 0): 16, (0, 0, 0, 1, 1, 0, 1, 1, 0): -24, (0, 0, 0, 1, 0, 1, 1, 1, 0): 16, (0, 0, 0, 0, 1, 1, 1, 1, 0): -3, (0, 0, 0, 0, 1, 1, 1, 0, 1): -4, (0, 0, 0, 0, 1, 1, 0, 1, 1): 6, (0, 0, 0, 0, 1, 0, 1, 1, 1): -4, (0, 0, 0, 0, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 0, 0, 0, 0): 1, (1, 1, 1, 0, 0, 1, 0, 0, 0): -1, (1, 1, 0, 1, 1, 0, 0, 0, 0): -3, (1, 1, 0, 1, 0, 1, 0, 0, 0): 3, (1, 0, 1, 1, 1, 0, 0, 0, 0): 2, (1, 0, 1, 1, 0, 1, 0, 0, 0): 0, (0, 1, 1, 1, 1, 0, 0, 0, 0): 0, (0, 1, 1, 1, 0, 1, 0, 0, 0): -5, (1, 0, 1, 0,

In [81]:
compare(4,5)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 0, 0, 0, 0): 1, (1, 1, 1, 1, 0, 1, 0, 0, 0): -5, (1, 1, 1, 0, 1, 1, 0, 0, 0): 10, (1, 1, 0, 1, 1, 1, 0, 0, 0): -10, (1, 0, 1, 1, 1, 1, 0, 0, 0): 5, (0, 1, 1, 1, 1, 1, 0, 0, 0): -4, (0, 1, 1, 1, 1, 0, 1, 0, 0): 15, (0, 1, 1, 1, 0, 1, 1, 0, 0): -30, (0, 1, 1, 0, 1, 1, 1, 0, 0): 30, (0, 1, 0, 1, 1, 1, 1, 0, 0): -15, (0, 0, 1, 1, 1, 1, 1, 0, 0): 6, (0, 0, 1, 1, 1, 1, 0, 1, 0): -15, (0, 0, 1, 1, 1, 0, 1, 1, 0): 30, (0, 0, 1, 1, 0, 1, 1, 1, 0): -30, (0, 0, 1, 0, 1, 1, 1, 1, 0): 15, (0, 0, 0, 1, 1, 1, 1, 1, 0): -4, (0, 0, 0, 1, 1, 1, 1, 0, 1): 5, (0, 0, 0, 1, 1, 1, 0, 1, 1): -10, (0, 0, 0, 1, 1, 0, 1, 1, 1): 10, (0, 0, 0, 1, 0, 1, 1, 1, 1): -5, (0, 0, 0, 0, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 0, 1, 0, 0, 0): 1, (1, 1, 1, 1, 0, 0, 1, 0, 0): -1, (1, 1, 1, 0, 1, 1, 0, 0, 0): -4, (1, 1, 1, 0, 1, 0, 1, 0, 0): 4, (1, 1, 0, 1, 1, 1, 0, 0, 0): 6, (1, 1, 0, 1, 1, 0, 1, 0, 0): -6, (1, 0, 1, 1, 1, 1, 0, 0, 0): -5, (1, 0, 1, 1, 1, 0, 1, 0, 0): 8, (0, 1, 1

In [83]:
compare(3,6)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 0, 0, 0): 1, (1, 1, 1, 1, 1, 0, 1, 0, 0): -6, (1, 1, 1, 1, 0, 1, 1, 0, 0): 15, (1, 1, 1, 0, 1, 1, 1, 0, 0): -20, (1, 1, 0, 1, 1, 1, 1, 0, 0): 15, (1, 0, 1, 1, 1, 1, 1, 0, 0): -6, (0, 1, 1, 1, 1, 1, 1, 0, 0): -1, (0, 1, 1, 1, 1, 1, 0, 1, 0): 12, (0, 1, 1, 1, 1, 0, 1, 1, 0): -30, (0, 1, 1, 1, 0, 1, 1, 1, 0): 40, (0, 1, 1, 0, 1, 1, 1, 1, 0): -30, (0, 1, 0, 1, 1, 1, 1, 1, 0): 12, (0, 0, 1, 1, 1, 1, 1, 1, 0): -1, (0, 0, 1, 1, 1, 1, 1, 0, 1): -6, (0, 0, 1, 1, 1, 1, 0, 1, 1): 15, (0, 0, 1, 1, 1, 0, 1, 1, 1): -20, (0, 0, 1, 1, 0, 1, 1, 1, 1): 15, (0, 0, 1, 0, 1, 1, 1, 1, 1): -6, (0, 0, 0, 1, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 1, 0, 1, 0, 0): 1, (1, 1, 1, 1, 1, 0, 0, 1, 0): -1, (1, 1, 1, 1, 0, 1, 1, 0, 0): -5, (1, 1, 1, 1, 0, 1, 0, 1, 0): 5, (1, 1, 1, 0, 1, 1, 1, 0, 0): 10, (1, 1, 1, 0, 1, 1, 0, 1, 0): -10, (1, 1, 0, 1, 1, 1, 1, 0, 0): -10, (1, 1, 0, 1, 1, 1, 0, 1, 0): 10, (1, 0, 1, 1, 1, 1, 1, 0, 0): 4, (1, 0, 1, 1, 1, 1, 0, 1, 0): 0, (0, 1

In [84]:
compare(2,7)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 1, 0, 0): 1, (1, 1, 1, 1, 1, 1, 0, 1, 0): -7, (1, 1, 1, 1, 1, 0, 1, 1, 0): 21, (1, 1, 1, 1, 0, 1, 1, 1, 0): -35, (1, 1, 1, 0, 1, 1, 1, 1, 0): 35, (1, 1, 0, 1, 1, 1, 1, 1, 0): -21, (1, 0, 1, 1, 1, 1, 1, 1, 0): 7, (0, 1, 1, 1, 1, 1, 1, 1, 0): -2, (0, 1, 1, 1, 1, 1, 1, 0, 1): 7, (0, 1, 1, 1, 1, 1, 0, 1, 1): -21, (0, 1, 1, 1, 1, 0, 1, 1, 1): 35, (0, 1, 1, 1, 0, 1, 1, 1, 1): -35, (0, 1, 1, 0, 1, 1, 1, 1, 1): 21, (0, 1, 0, 1, 1, 1, 1, 1, 1): -7, (0, 0, 1, 1, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 1, 1, 0, 1, 0): 1, (1, 1, 1, 1, 1, 1, 0, 0, 1): -1, (1, 1, 1, 1, 1, 0, 1, 1, 0): -6, (1, 1, 1, 1, 1, 0, 1, 0, 1): 6, (1, 1, 1, 1, 0, 1, 1, 1, 0): 15, (1, 1, 1, 1, 0, 1, 1, 0, 1): -15, (1, 1, 1, 0, 1, 1, 1, 1, 0): -20, (1, 1, 1, 0, 1, 1, 1, 0, 1): 20, (1, 1, 0, 1, 1, 1, 1, 1, 0): 15, (1, 1, 0, 1, 1, 1, 1, 0, 1): -15, (1, 0, 1, 1, 1, 1, 1, 1, 0): -7, (1, 0, 1, 1, 1, 1, 1, 0, 1): 12, (0, 1, 1, 1, 1, 1, 1, 1, 0): 2, (0, 1, 1, 1, 1, 1, 1, 0, 1): -7, (1, 0

In [85]:
compare(1,8)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 1, 1, 0): 1, (1, 1, 1, 1, 1, 1, 1, 0, 1): -8, (1, 1, 1, 1, 1, 1, 0, 1, 1): 28, (1, 1, 1, 1, 1, 0, 1, 1, 1): -56, (1, 1, 1, 1, 0, 1, 1, 1, 1): 70, (1, 1, 1, 0, 1, 1, 1, 1, 1): -56, (1, 1, 0, 1, 1, 1, 1, 1, 1): 28, (1, 0, 1, 1, 1, 1, 1, 1, 1): -8, (0, 1, 1, 1, 1, 1, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 1, 1, 1, 1, 1, 2): -8, (1, 1, 1, 1, 1, 1, 2, 1): 28, (1, 1, 1, 1, 1, 2, 1, 1): -56, (1, 1, 1, 1, 2, 1, 1, 1): 70, (1, 1, 1, 2, 1, 1, 1, 1): -56, (1, 1, 2, 1, 1, 1, 1, 1): 28, (1, 2, 1, 1, 1, 1, 1, 1): -8, (2, 1, 1, 1, 1, 1, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1, 1, 1, 1, 1, 1): -1, (1, 2, 1, 1, 1, 1, 1, 1): 7, (1, 1, 2, 1, 1, 1, 1, 1): -21, (1, 1, 1, 2, 1, 1, 1, 1): 35, (1, 1, 1, 1, 2, 1, 1, 1): -35, (1, 1, 1, 1, 1, 2, 1, 1): 21, (1, 1, 1, 1, 1, 1, 2, 1): -7, (1, 1, 1, 1, 1, 1, 1, 2): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[ -8  28 -56  70 -56  28  -8   1]
Matrix of pr

In [86]:
compare(8,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 6, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0): -14, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0): -14, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0): 42, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0): 14, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0): -70, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0): 0, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0): 70, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0): -14, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0): -42, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0): 14, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0): 14, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0): -6, (0, 0, 0, 0, 0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 0, 1, 0, 0, 0, 0, 0, 0): 3, (1, 0, 0, 0, 1, 0, 0, 0, 0, 0): -2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 1, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0): 1, (0, 1, 0, 0, 1, 0, 0, 0, 0, 0): -12, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0): -4, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0): 9, (0, 1, 0, 0, 0, 1, 0, 0, 0, 0): 10, (0, 0, 1, 0, 0, 1, 0, 0, 0, 

In [89]:
compare(6,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0, 0, 0, 0, 0, 0): -1, (1, 1, 1, 0, 1, 0, 0, 0, 0, 0): 4, (1, 1, 0, 1, 1, 0, 0, 0, 0, 0): -6, (1, 0, 1, 1, 1, 0, 0, 0, 0, 0): 4, (0, 1, 1, 1, 1, 0, 0, 0, 0, 0): 4, (0, 1, 1, 1, 0, 1, 0, 0, 0, 0): -20, (0, 1, 1, 0, 1, 1, 0, 0, 0, 0): 30, (0, 1, 0, 1, 1, 1, 0, 0, 0, 0): -20, (0, 0, 1, 1, 1, 1, 0, 0, 0, 0): -5, (0, 0, 1, 1, 1, 0, 1, 0, 0, 0): 40, (0, 0, 1, 1, 0, 1, 1, 0, 0, 0): -60, (0, 0, 1, 0, 1, 1, 1, 0, 0, 0): 40, (0, 0, 0, 1, 1, 1, 1, 0, 0, 0): 0, (0, 0, 0, 1, 1, 1, 0, 1, 0, 0): -40, (0, 0, 0, 1, 1, 0, 1, 1, 0, 0): 60, (0, 0, 0, 1, 0, 1, 1, 1, 0, 0): -40, (0, 0, 0, 0, 1, 1, 1, 1, 0, 0): 5, (0, 0, 0, 0, 1, 1, 1, 0, 1, 0): 20, (0, 0, 0, 0, 1, 1, 0, 1, 1, 0): -30, (0, 0, 0, 0, 1, 0, 1, 1, 1, 0): 20, (0, 0, 0, 0, 0, 1, 1, 1, 1, 0): -4, (0, 0, 0, 0, 0, 1, 1, 1, 0, 1): -4, (0, 0, 0, 0, 0, 1, 1, 0, 1, 1): 6, (0, 0, 0, 0, 0, 1, 0, 1, 1, 1): -4, (0, 0, 0, 0, 0, 0, 1, 1, 1, 1): 1}, {(1, 1, 1, 0, 1, 0, 0, 0, 0, 0): -1, (1, 1, 1, 0, 0, 1, 0, 0, 0,

In [90]:
compare(4,6)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 0, 0, 0, 0): -1, (1, 1, 1, 1, 1, 0, 1, 0, 0, 0): 6, (1, 1, 1, 1, 0, 1, 1, 0, 0, 0): -15, (1, 1, 1, 0, 1, 1, 1, 0, 0, 0): 20, (1, 1, 0, 1, 1, 1, 1, 0, 0, 0): -15, (1, 0, 1, 1, 1, 1, 1, 0, 0, 0): 6, (0, 1, 1, 1, 1, 1, 1, 0, 0, 0): 2, (0, 1, 1, 1, 1, 1, 0, 1, 0, 0): -18, (0, 1, 1, 1, 1, 0, 1, 1, 0, 0): 45, (0, 1, 1, 1, 0, 1, 1, 1, 0, 0): -60, (0, 1, 1, 0, 1, 1, 1, 1, 0, 0): 45, (0, 1, 0, 1, 1, 1, 1, 1, 0, 0): -18, (0, 0, 1, 1, 1, 1, 1, 1, 0, 0): 0, (0, 0, 1, 1, 1, 1, 1, 0, 1, 0): 18, (0, 0, 1, 1, 1, 1, 0, 1, 1, 0): -45, (0, 0, 1, 1, 1, 0, 1, 1, 1, 0): 60, (0, 0, 1, 1, 0, 1, 1, 1, 1, 0): -45, (0, 0, 1, 0, 1, 1, 1, 1, 1, 0): 18, (0, 0, 0, 1, 1, 1, 1, 1, 1, 0): -2, (0, 0, 0, 1, 1, 1, 1, 1, 0, 1): -6, (0, 0, 0, 1, 1, 1, 1, 0, 1, 1): 15, (0, 0, 0, 1, 1, 1, 0, 1, 1, 1): -20, (0, 0, 0, 1, 1, 0, 1, 1, 1, 1): 15, (0, 0, 0, 1, 0, 1, 1, 1, 1, 1): -6, (0, 0, 0, 0, 1, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 1, 0, 1, 0, 0, 0): -1, (1, 1, 1, 1, 1, 0, 0, 1

In [91]:
compare(2,8)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 1, 1, 1, 1, 0, 0): -1, (1, 1, 1, 1, 1, 1, 1, 0, 1, 0): 8, (1, 1, 1, 1, 1, 1, 0, 1, 1, 0): -28, (1, 1, 1, 1, 1, 0, 1, 1, 1, 0): 56, (1, 1, 1, 1, 0, 1, 1, 1, 1, 0): -70, (1, 1, 1, 0, 1, 1, 1, 1, 1, 0): 56, (1, 1, 0, 1, 1, 1, 1, 1, 1, 0): -28, (1, 0, 1, 1, 1, 1, 1, 1, 1, 0): 8, (0, 1, 1, 1, 1, 1, 1, 1, 1, 0): 0, (0, 1, 1, 1, 1, 1, 1, 1, 0, 1): -8, (0, 1, 1, 1, 1, 1, 1, 0, 1, 1): 28, (0, 1, 1, 1, 1, 1, 0, 1, 1, 1): -56, (0, 1, 1, 1, 1, 0, 1, 1, 1, 1): 70, (0, 1, 1, 1, 0, 1, 1, 1, 1, 1): -56, (0, 1, 1, 0, 1, 1, 1, 1, 1, 1): 28, (0, 1, 0, 1, 1, 1, 1, 1, 1, 1): -8, (0, 0, 1, 1, 1, 1, 1, 1, 1, 1): 1}, {(1, 1, 1, 1, 1, 1, 1, 0, 1, 0): -1, (1, 1, 1, 1, 1, 1, 1, 0, 0, 1): 1, (1, 1, 1, 1, 1, 1, 0, 1, 1, 0): 7, (1, 1, 1, 1, 1, 1, 0, 1, 0, 1): -7, (1, 1, 1, 1, 1, 0, 1, 1, 1, 0): -21, (1, 1, 1, 1, 1, 0, 1, 1, 0, 1): 21, (1, 1, 1, 1, 0, 1, 1, 1, 1, 0): 35, (1, 1, 1, 1, 0, 1, 1, 1, 0, 1): -35, (1, 1, 1, 0, 1, 1, 1, 1, 1, 0): -35, (1, 1, 1, 0, 1, 1, 1, 1,

In [87]:
compare(10,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 8, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0): -18, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0): -27, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0): 72, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 48, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0): -168, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0): -42, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0): 252, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0): 0, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0): -252, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0): 42, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0): 168, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0): -48, (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0): -72, (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0): 27, (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0): 18, (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0): -8, (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1): -2, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [88]:
compare(12,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 10, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -22, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -44, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 110, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 110, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0): -330, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0): -165, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0): 660, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 132, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0): -924, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0): 0, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0): 924, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0): -132, (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0): -660, (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0): 165, (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0): 330, (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0): -110, (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [102]:
compare(14,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 12, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -26, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -65, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 156, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 208, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -572, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -429, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1430, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 572, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0): -2574, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0): -429, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0): 3432, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 0, (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0): -3432, (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0): 429, (0, 0, 0, 0, 0, 0, 0, 0, 1

In [103]:
compare(16,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 14, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -30, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -90, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 210, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 350, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -910, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -910, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2730, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1638, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -6006, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -2002, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 10010, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1430, (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,

In [105]:
compare(17,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -15, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 32, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 104, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -240, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -440, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1120, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 1260, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -3640, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -2548, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 8736, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 3640, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -16016, (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 

In [104]:
compare(18,2) #floor((wt-2)/6)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1, (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 2, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 16, (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -34, (0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -119, (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 272, (0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 544, (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1360, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -1700, (0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 4760, (0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 3808, (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -12376, (0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): -6188, (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 24752, (0, 0, 0, 0, 0, 

In [98]:
find_nonzero_kernel(10)

(nx, ny) = (1, 9):
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (2, 8):
Free module of degree 8 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (3, 7):
Free module of degree 24 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (4, 6):
Free module of degree 40 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (5, 5):
Free module of degree 50 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (6, 4):
Free module of degree 40 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (7, 3):
Free module of degree 24 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (8, 2):
Free module of degree 8 and rank 1 over Integer Ring
Echelon basis matrix:
[ 0  2 -1 -1  0  2 -7  7]
Computation time: 0.1540088653564453


In [100]:
find_nonzero_kernel(11)

(nx, ny) = (1, 10):
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (2, 9):
Free module of degree 10 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (3, 8):
Free module of degree 30 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (4, 7):
Free module of degree 60 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (5, 6):
Free module of degree 84 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (6, 5):
Free module of degree 84 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (7, 4):
Free module of degree 60 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (8, 3):
Free module of degree 30 and rank 1 over Integer Ring
Echelon basis matrix:
[  0   2   2  -1  -6   5  -1  -4  -2   2   0  -2  -1   2  -1   0  -2   5   2 -10   4  10  -4  -4 -24  30  30 -36  -5 -20]
(nx, ny) = (9, 2):
Free module of degree 10 and rank 0 over Integer Ring
Echelon basis matrix:
[]

In [101]:
find_nonzero_kernel(12)

(nx, ny) = (1, 11):
Free module of degree 2 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (2, 10):
Free module of degree 10 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (3, 9):
Free module of degree 36 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (4, 8):
Free module of degree 80 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (5, 7):
Free module of degree 132 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (6, 6):
Free module of degree 150 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (7, 5):
Free module of degree 132 and rank 0 over Integer Ring
Echelon basis matrix:
[]
(nx, ny) = (8, 4):
Free module of degree 80 and rank 1 over Integer Ring
Echelon basis matrix:
[   0    1    0    1    2    7   14    7    7    3    3   -8    0  -31  -10  -25   -1  -33  -51   -1    0    0   -8   12  -27   20    6    0   -5    1   -1    8    0   -1   -5   -7   -3    0    6   28    0    1    0

In [3]:
is_lyndon((1,1,1,2,1,2))

True

In [8]:
compare(4,1)

primitive elements of xy-shuffle:
[{(1, 0, 0, 0, 0): 1, (0, 1, 0, 0, 0): -4, (0, 0, 1, 0, 0): 6, (0, 0, 0, 1, 0): -4, (0, 0, 0, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(5,): 1}]
primitive elements of z-value-shuffle:
[{(5,): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[1]
Matrix of primitive elements of z-value-shuffle:
[1]
Kernel:
Free module of degree 2 and rank 1 over Integer Ring
Echelon basis matrix:
[ 1 -1]


In [9]:
compare(5,1)

primitive elements of xy-shuffle:
[{(1, 0, 0, 0, 0, 0): -1, (0, 1, 0, 0, 0, 0): 5, (0, 0, 1, 0, 0, 0): -10, (0, 0, 0, 1, 0, 0): 10, (0, 0, 0, 0, 1, 0): -5, (0, 0, 0, 0, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(6,): 1}]
primitive elements of z-value-shuffle:
[{(6,): 1}]
Matrix of pi_y( primitive elements of xy-shuffle ):
[1]
Matrix of primitive elements of z-value-shuffle:
[1]
Kernel:
Free module of degree 2 and rank 1 over Integer Ring
Echelon basis matrix:
[ 1 -1]
